In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import os

In [ ]:
main_dir = './archive/New Masks Dataset'
train_dir = os.path.join(main_dir,'Train')
test_dir = os.path.join(main_dir,'Test')
valid_dir = os.path.join(main_dir, 'Validation')

train_mask_dir = os.path.join(train_dir,'Mask')
train_nomask_dir = os.path.join(train_dir,'Non Mask')

In [ ]:
train_mask_names = os.listdir(train_mask_dir)
train_nomask_names = os.listdir(train_nomask_dir)
print(train_mask_names[:10])
print(train_nomask_names[:10])

In [ ]:
import matplotlib.image as mpimg
nrows = 4
ncols = 4
plt.figure(figsize=(12,12))

mask_pic = []
for i in train_mask_names[0:8]:
    mask_pic.append(os.path.join(train_mask_dir,i))

nomask_pic = []
for i in train_nomask_names[0:8]:
    nomask_pic.append(os.path.join(train_nomask_dir,i))

print(mask_pic)
print(nomask_pic)

merged_list = mask_pic+nomask_pic

for i in range(0,len(merged_list)):
    data = merged_list[i].split('\\',2)[2]
    print(data)
    sp = plt.subplot(nrows,ncols,i+1)
    sp.axis('Off')
    image = mpimg.imread(merged_list[i])
    sp.set_title(data,fontsize=10)
    plt.imshow(image,cmap='gray')

plt.show()

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,zoom_range = 0.2,rotation_range = 40,horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale=1./255)

validation_datagen  = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,target_size = (150,150),batch_size = 32,class_mode = 'binary')

test_generator = test_datagen.flow_from_directory(test_dir,target_size = (150,150),batch_size = 32,class_mode = 'binary')

validation_generator = validation_datagen.flow_from_directory(valid_dir,target_size = (150,150),batch_size = 32,class_mode = 'binary')

In [ ]:
validation_generator.class_indices

In [ ]:
train_generator.image_shape

In [ ]:
model = Sequential()

model.add(Conv2D(32,(3,3), padding='SAME',activation='relu',input_shape=(150,150,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))


model.add(Conv2D(64,(3,3), padding='SAME',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1,activation='sigmoid'))
model.summary()

In [ ]:
model.compile(Adam(lr=0.001),loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator, epochs= 30, validation_data = validation_generator)

In [ ]:
history.history.keys()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['training','validation'])
plt.title('Training and Validation loss')
plt.xlabel('epoch')

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['training','validation'])
plt.title('Training and Validation accuracy')
plt.xlabel('epoch')

In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print('Test loss : '+str(test_loss)+'\n'+'Test accuracy : '+str(test_acc))

In [ ]:
test_image_downloaded = './Test images'
os.listdir(test_image_downloaded)

In [ ]:
from keras.preprocessing import image

In [ ]:
for fname in os.listdir(test_image_downloaded):
    img_path = './Test images/'+fname
    img = image.load_img(img_path,target_size=(150,150))
    images = image.img_to_array(img)
    images = np.expand_dims(images,axis=0)
    prediction = model.predict(images)
    print(fname + ' image is predicted as ' , end='')
    if prediction == 0:
        print('Masked Image')
    else:
        print('Not Masked Image')

In [ ]:
model.save('model.h5')